# 第 9 章: Change Data Capture によるリアルタイムデータ同期

このノートブックでは、CDC（Change Data Capture）ログを使用して Iceberg テーブルにデータを同期する方法を学びます。

### 注意事項
本ハンズオンでは、CDC パイプライン全体の構築は行いません。すでに CDC ログ（Debezium 形式）が生成されているという前提で進めます。実際の運用環境では Debezium + Kafka + Kafka Connect や Flink などを使用して CDC ログを生成しますが、ここでは CDC ログの処理と Iceberg テーブルへの適用方法に焦点を当てます。

## 環境の準備

In [68]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime
from decimal import Decimal
import pandas as pd

CATALOG = "my_catalog"
CATALOG_URL = "http://server:8181/"
S3_ENDPOINT = "http://minio:9000"
SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.8.1"

In [69]:
# SparkSessionの初期化
spark = (
    SparkSession.builder
        .config("spark.jars.packages", f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}")
        .config(f"spark.sql.catalog.{CATALOG}", "org.apache.iceberg.spark.SparkCatalog")
        .config(f"spark.sql.catalog.{CATALOG}.type", "rest")
        .config(f"spark.sql.catalog.{CATALOG}.uri", CATALOG_URL)
        .config(f"spark.sql.catalog.{CATALOG}.s3.endpoint", S3_ENDPOINT)
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.defaultCatalog", CATALOG)
        .getOrCreate()
)

In [70]:
%sql spark

In [71]:
%%sql
CREATE DATABASE IF NOT EXISTS db

Running query in 'SparkSession'

""


## シナリオ: ECサイトの商品管理システム

ECサイトの商品管理システムを想定します。商品の在庫管理テーブルに対する1時間分のCDCログ（Debezium形式）がすでに生成されており、これをIcebergテーブルに反映させます。CDCログには、新商品の追加、在庫数の更新、廃盤商品の削除などが含まれています。

### 1. Iceberg テーブルの作成と初期データの投入

In [72]:
%%sql
CREATE OR REPLACE TABLE db.products (
    product_id BIGINT,
    product_name STRING,
    category STRING,
    price DECIMAL(10,2),
    stock_quantity INT,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
) USING iceberg
PARTITIONED BY (category)

Running query in 'SparkSession'

""


In [73]:
# 初期データの投入
initial_products = [
    {
        'product_id': 2001, 
        'product_name': 'Wireless Headphones', 
        'category': 'Electronics',
        'price': Decimal('89.99'), 
        'stock_quantity': 150,
        'created_at': datetime(2025, 1, 15, 10, 30),
        'updated_at': datetime(2025, 1, 15, 10, 30)
    },
    {
        'product_id': 2002, 
        'product_name': 'USB Cable', 
        'category': 'Accessories',
        'price': Decimal('12.99'), 
        'stock_quantity': 200,
        'created_at': datetime(2025, 1, 15, 11, 0),
        'updated_at': datetime(2025, 1, 15, 11, 0)
    },
    {
        'product_id': 2003, 
        'product_name': 'Bluetooth Speaker', 
        'category': 'Electronics',
        'price': Decimal('45.99'), 
        'stock_quantity': 75,
        'created_at': datetime(2025, 1, 15, 12, 0),
        'updated_at': datetime(2025, 1, 15, 12, 0)
    }
]

df_initial = spark.createDataFrame(initial_products)
df_initial.writeTo('db.products').append()

In [74]:
%%sql
SELECT * FROM db.products ORDER BY product_id

Running query in 'SparkSession'

3 rows affected.

,product_id,product_name,category,price,stock_quantity,created_at,updated_at
0,2001,Wireless Headphones,Electronics,89.99,150,2025-01-15 10:30:00,2025-01-15 10:30:00
1,2002,USB Cable,Accessories,12.99,200,2025-01-15 11:00:00,2025-01-15 11:00:00
2,2003,Bluetooth Speaker,Electronics,45.99,75,2025-01-15 12:00:00,2025-01-15 12:00:00


### 2. Debezium 形式の CDC イベントの準備

Debezium は変更イベントを構造化された形式で出力します。各イベントには以下の情報が含まれます：
- `payload.before`: 変更前の値（UPDATE, DELETE時）
- `payload.after`: 変更後の値（INSERT, UPDATE時）
- `payload.op`: 操作タイプ（c: create/insert, u: update, d: delete）
- `payload.source`: メタデータ（タイムスタンプ、ログ位置など）

In [75]:
# Debezium形式のCDCイベントサンプル
import json
import tempfile
import os

# CDCイベントをJSON形式で定義
cdc_events_json = [
    # 新商品の追加（INSERT）
    {
        "payload": {
            "before": None,
            "after": {
                "product_id": 2004,
                "product_name": "Gaming Mouse",
                "category": "Electronics",
                "price": 59.99,
                "stock_quantity": 100,
                "created_at": "2025-01-15T14:30:00Z",
                "updated_at": "2025-01-15T14:30:00Z"
            },
            "op": "c",
            "ts_ms": 1737037800000,
            "source": {
                "db": "inventory",
                "table": "products",
                "ts_ms": 1737037800000
            }
        }
    },
    
    # 既存商品の在庫数更新（UPDATE）
    {
        "payload": {
            "before": {
                "product_id": 2001,
                "product_name": "Wireless Headphones",
                "category": "Electronics",
                "price": 89.99,
                "stock_quantity": 150,
                "created_at": "2025-01-15T10:30:00Z",
                "updated_at": "2025-01-15T10:30:00Z"
            },
            "after": {
                "product_id": 2001,
                "product_name": "Wireless Headphones",
                "category": "Electronics",
                "price": 89.99,
                "stock_quantity": 143,  # 在庫が7個減少
                "created_at": "2025-01-15T10:30:00Z",
                "updated_at": "2025-01-15T15:00:00Z"
            },
            "op": "u",
            "ts_ms": 1737039600000,
            "source": {
                "db": "inventory",
                "table": "products",
                "ts_ms": 1737039600000
            }
        }
    },
    
    # 廃盤商品の削除（DELETE）
    {
        "payload": {
            "before": {
                "product_id": 2002,
                "product_name": "USB Cable",
                "category": "Accessories",
                "price": 12.99,
                "stock_quantity": 200,
                "created_at": "2025-01-15T11:00:00Z",
                "updated_at": "2025-01-15T11:00:00Z"
            },
            "after": None,
            "op": "d",
            "ts_ms": 1737041400000,
            "source": {
                "db": "inventory",
                "table": "products",
                "ts_ms": 1737041400000
            }
        }
    }
]

# 一時ファイルにJSONとして保存
temp_dir = tempfile.mkdtemp()
json_file_path = os.path.join(temp_dir, "cdc_events.json")

with open(json_file_path, 'w') as f:
    for event in cdc_events_json:
        f.write(json.dumps(event) + '\n')

print(f"CDC イベント数: {len(cdc_events_json)}")
print("- INSERT: 1件 (product_id: 2004 - Gaming Mouse)")
print("- UPDATE: 1件 (product_id: 2001 - 在庫数更新)")
print("- DELETE: 1件 (product_id: 2002 - USB Cable削除)")
print(f"JSONファイル保存先: {json_file_path}")

CDC イベント数: 3
- INSERT: 1件 (product_id: 2004 - Gaming Mouse)
- UPDATE: 1件 (product_id: 2001 - 在庫数更新)
- DELETE: 1件 (product_id: 2002 - USB Cable削除)
JSONファイル保存先: /tmp/tmpnusdo93e/cdc_events.json


### 3. CDC イベントの変換処理

CDCイベントを処理用に変換します。この処理では、操作タイプに応じて適切なフィールドから値を抽出し、統一された形式に変換します。

In [76]:
# JSONファイルからCDCイベントを読み込み
df_cdc_raw = spark.read.json(json_file_path)
print("CDCイベントの構造:")
df_cdc_raw.printSchema()
print("\nCDCイベントの内容:")
df_cdc_raw.show(truncate=False)

CDCイベントの構造:
root
 |-- payload: struct (nullable = true)
 |    |-- after: struct (nullable = true)
 |    |    |-- category: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- price: double (nullable = true)
 |    |    |-- product_id: long (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    |-- stock_quantity: long (nullable = true)
 |    |    |-- updated_at: string (nullable = true)
 |    |-- before: struct (nullable = true)
 |    |    |-- category: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- price: double (nullable = true)
 |    |    |-- product_id: long (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    |-- stock_quantity: long (nullable = true)
 |    |    |-- updated_at: string (nullable = true)
 |    |-- op: string (nullable = true)
 |    |-- source: struct (nullable = true)
 |    |    |-- db: string (nullable = true)
 |    |    |-- 

In [77]:
# スキーマを明示的に定義してCDCイベントを処理
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType

# Debeziumのスキーマ定義
product_schema = StructType([
    StructField("product_id", LongType(), True),
    StructField("product_name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("stock_quantity", LongType(), True),
    StructField("created_at", StringType(), True),
    StructField("updated_at", StringType(), True)
])

source_schema = StructType([
    StructField("db", StringType(), True),
    StructField("table", StringType(), True),
    StructField("ts_ms", LongType(), True)
])

payload_schema = StructType([
    StructField("before", product_schema, True),
    StructField("after", product_schema, True),
    StructField("op", StringType(), True),
    StructField("ts_ms", LongType(), True),
    StructField("source", source_schema, True)
])

cdc_schema = StructType([
    StructField("payload", payload_schema, True)
])

# スキーマを指定してJSONファイルを読み込み
df_cdc_structured = spark.read.schema(cdc_schema).json(json_file_path)

print("構造化されたCDCイベントのスキーマ:")
df_cdc_structured.printSchema()

# CDCイベントを処理用に変換
df_changes = df_cdc_structured.select(
    # 削除操作の場合はbeforeから、それ以外はafterからproduct_idを取得
    F.when(F.col("payload.op") == "d", F.col("payload.before.product_id"))
     .otherwise(F.col("payload.after.product_id")).alias("product_id"),
    
    # 削除操作以外の場合はafterから値を取得
    F.col("payload.after.product_name").alias("product_name"),
    F.col("payload.after.category").alias("category"),
    F.col("payload.after.price").cast("decimal(10,2)").alias("price"),
    F.col("payload.after.stock_quantity").alias("stock_quantity"),
    F.to_timestamp(F.col("payload.after.created_at")).alias("created_at"),
    F.to_timestamp(F.col("payload.after.updated_at")).alias("updated_at"),
    
    # メタデータ
    F.col("payload.op").alias("_op"),
    F.col("payload.ts_ms").alias("_ts_ms"),
    F.from_unixtime(F.col("payload.ts_ms") / 1000).cast("timestamp").alias("_change_time")
)

print("\n変換後のCDCデータ:")
df_changes.show(truncate=False)

構造化されたCDCイベントのスキーマ:
root
 |-- payload: struct (nullable = true)
 |    |-- before: struct (nullable = true)
 |    |    |-- product_id: long (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    |-- category: string (nullable = true)
 |    |    |-- price: double (nullable = true)
 |    |    |-- stock_quantity: long (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- updated_at: string (nullable = true)
 |    |-- after: struct (nullable = true)
 |    |    |-- product_id: long (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    |-- category: string (nullable = true)
 |    |    |-- price: double (nullable = true)
 |    |    |-- stock_quantity: long (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- updated_at: string (nullable = true)
 |    |-- op: string (nullable = true)
 |    |-- ts_ms: long (nullable = true)
 |    |-- source: struct (nullable = true)
 |    |    

### 4. 同一レコードの複数変更への対応

同一商品に対する複数の変更がある場合、最新の変更のみを適用します。

In [78]:
# 各product_idごとに最新の変更のみを抽出
window = Window.partitionBy("product_id").orderBy(F.desc("_ts_ms"))
df_latest = df_changes.withColumn("rank", F.row_number().over(window)) \
                     .filter(F.col("rank") == 1) \
                     .drop("rank")

print("重複除去後のCDCデータ:")
df_latest.show(truncate=False)

# ステージングテーブルとして一時ビューを作成
df_latest.createOrReplaceTempView("staging_changes")

重複除去後のCDCデータ:
+----------+-------------------+-----------+-----+--------------+-------------------+-------------------+---+-------------+-------------------+
|product_id|product_name       |category   |price|stock_quantity|created_at         |updated_at         |_op|_ts_ms       |_change_time       |
+----------+-------------------+-----------+-----+--------------+-------------------+-------------------+---+-------------+-------------------+
|2001      |Wireless Headphones|Electronics|89.99|143           |2025-01-15 10:30:00|2025-01-15 15:00:00|u  |1737039600000|2025-01-16 15:00:00|
|2002      |NULL               |NULL       |NULL |NULL          |NULL               |NULL               |d  |1737041400000|2025-01-16 15:30:00|
|2004      |Gaming Mouse       |Electronics|59.99|100           |2025-01-15 14:30:00|2025-01-15 14:30:00|c  |1737037800000|2025-01-16 14:30:00|
+----------+-------------------+-----------+-----+--------------+-------------------+-------------------+---+-------------

### 5. MERGE INTO 文による Iceberg テーブルの更新

In [79]:
%%sql
MERGE INTO db.products AS target
USING staging_changes AS source
ON target.product_id = source.product_id
WHEN MATCHED AND source._op = 'd' THEN
    DELETE
WHEN MATCHED AND source._op IN ('u', 'c') THEN
    UPDATE SET
        target.product_name = source.product_name,
        target.category = source.category,
        target.price = source.price,
        target.stock_quantity = source.stock_quantity,
        target.created_at = source.created_at,
        target.updated_at = source.updated_at
WHEN NOT MATCHED AND source._op IN ('c', 'u') THEN
    INSERT (product_id, product_name, category, price, stock_quantity, created_at, updated_at)
    VALUES (source.product_id, source.product_name, source.category, 
            source.price, source.stock_quantity, source.created_at, source.updated_at)

Running query in 'SparkSession'

""


### 6. 結果の確認

In [80]:
%%sql
SELECT * FROM db.products

Running query in 'SparkSession'

3 rows affected.

,product_id,product_name,category,price,stock_quantity,created_at,updated_at
0,2001,Wireless Headphones,Electronics,89.99,143,2025-01-15 10:30:00,2025-01-15 15:00:00
1,2003,Bluetooth Speaker,Electronics,45.99,75,2025-01-15 12:00:00,2025-01-15 12:00:00
2,2004,Gaming Mouse,Electronics,59.99,100,2025-01-15 14:30:00,2025-01-15 14:30:00


## まとめ

このハンズオンでは、CDC（Change Data Capture）ログを使用して Iceberg テーブルにデータを同期する方法を学びました：

1. **Debezium形式のCDCイベント**: 変更前後の値と操作タイプ（INSERT/UPDATE/DELETE）を含む構造化されたイベント
2. **データ変換処理**: CDCイベントを処理用の統一された形式に変換
3. **重複排除**: 同一レコードの複数回更新に対する最新状態の抽出
4. **MERGE INTO文**: CDCデータをIcebergテーブルに適用する強力なSQL文
5. **バッチ処理関数**: 実運用を想定した処理の関数化

実際の運用では、Debezium + Kafka + Kafka Connect や Flink などを使用してリアルタイムでCDCログを処理しますが、基本的な処理パターンは本ハンズオンで学んだ内容と同じです。

このような処理を定期的に実行することで、ソースシステムの変更をIcebergテーブルに効率的に同期できます。